In [ ]:
__author__ = "Casper Kristiansson"
__copyright__ = "WeatherBrain"

__maintainer__ = "Casper Kristiansson"
__email__ = "casperkr@kth.se"
__status__ = "Development"

In [80]:
import pandas

dataframe_temperature = pandas.read_csv('..\\Data\\Processed Data\\Temperature.csv')
dataframe_hpa = pandas.read_csv('..\\Data\\Processed Data\\hPa.csv')

print(dataframe_temperature.shape)
print(dataframe_temperature)

(95694, 6)
                      date  temperature_one  temperature_two  \
0      1756-01-01 00:00:00             -8.0             -9.0   
1      1756-01-02 00:00:00             -9.0             -9.0   
2      1756-01-03 00:00:00            -10.0             -7.0   
3      1756-01-04 00:00:00             -7.0             -8.0   
4      1756-01-05 00:00:00             -9.0             -5.0   
...                    ...              ...              ...   
95689  2017-12-27 00:00:00              2.9              3.5   
95690  2017-12-28 00:00:00              4.3              4.9   
95691  2017-12-29 00:00:00              2.9              2.7   
95692  2017-12-30 00:00:00              1.4              1.4   
95693  2017-12-31 00:00:00              2.5              3.2   

       temperature_three  temperature_four  temperature_five  
0                    NaN               NaN               NaN  
1                    NaN               NaN               NaN  
2                    NaN       

In [81]:
row_hpa = dataframe_hpa.iloc[10000]
mean_hpa = np.nanmean([row_hpa['hPa_one'], row_hpa['hPa_two'], row_hpa['hPa_three']])
print(mean_hpa, [row_hpa['hPa_one'], row_hpa['hPa_two'], row_hpa['hPa_three']])

1020.15 [1018.5, nan, 1021.8]


In [ ]:
import math
import datetime
import numpy as np

result = []

for index, row in dataframe_temperature.iterrows():
    time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    data = []
    length = 0
    mean = 0
    for value in row:
        if type(value) == float and not math.isnan(value):
            length += 1
            mean += value

    if length == 0:
        continue
    
    mean = mean / length
    time_difference = 24 / length

    row_hpa = dataframe_hpa.iloc[index]

    if all(np.isnan(x) for x in [row_hpa['hPa_one'], row_hpa['hPa_two'], row_hpa['hPa_three']]):
        mean_hpa = np.nan
    else:
        mean_hpa = np.nanmean([row_hpa['hPa_one'], row_hpa['hPa_two'], row_hpa['hPa_three']])
    
    for value in row:
        if type(value) == float and not math.isnan(value):
            data.append(
                [time, value, mean, mean_hpa]
            )
            time += datetime.timedelta(hours=time_difference)

    result.append(
        pandas.DataFrame(data, columns=['Date', 'Temperature', 'Mean Temperature', 'Mean hPa'])
    )

dataframe_result = pandas.concat(result)

In [79]:
print(dataframe_result)
dataframe_result.to_csv('..\\Data\\Processed Data\\Combined Cleaned.csv', index=False)

                  Date  Temperature  Mean Temperature     Mean hPa
0  1756-01-01 00:00:00         -8.0              -8.5  1003.000000
1  1756-01-01 12:00:00         -9.0              -8.5  1003.000000
0  1756-01-02 00:00:00         -9.0              -9.0  1009.800000
1  1756-01-02 12:00:00         -9.0              -9.0  1009.800000
0  1756-01-03 00:00:00        -10.0              -8.5  1012.650000
..                 ...          ...               ...          ...
0  2017-12-31 00:00:00          2.5               2.9   997.866667
1  2017-12-31 04:48:00          3.2               2.9   997.866667
2  2017-12-31 09:36:00          3.1               2.9   997.866667
3  2017-12-31 14:24:00          3.4               2.9   997.866667
4  2017-12-31 19:12:00          2.3               2.9   997.866667

[401113 rows x 4 columns]


In [65]:
dataframe_result = pandas.read_csv('..\\Data\\Processed Data\\Temperature_cleaned.csv')
dataframe_hpa = pandas.read_csv('..\\Data\\Processed Data\\hPa.csv')

print(dataframe_result.columns)
print(dataframe_hpa.columns)

Index(['Date', 'Temperature', 'Mean'], dtype='object')
Index(['date', 'hPa_one', 'hPa_two', 'hPa_three'], dtype='object')


In [66]:
import datetime

for index, row in dataframe_result.iterrows():
    dataframe_result.at[index, 'Date'] = datetime.datetime.strptime(row['Date'], '%Y-%m-%d %H:%M:%S')

for index, row in dataframe_hpa.iterrows():
    dataframe_hpa.at[index, 'date'] = datetime.datetime.strptime(row['date'], '%Y-%m-%d').date()

In [ ]:
import datetime
result = []

for index_hpa, row_hpa in dataframe_hpa.iterrows():
    mean = (row_hpa['hPa_one'] + row_hpa['hPa_two'] + row_hpa['hPa_three']) / 3
    search = False
    for index_temp, row_temp in dataframe_result.iterrows():
        if row_temp['Date'].year == row_hpa['date'].year and row_temp['Date'].month == row_hpa['date'].month and row_temp['Date'].day == row_hpa['date'].day:
            search = True
            result.append(
                pandas.DataFrame([[row_temp['Date'], row_temp['Temperature'], row_temp['Mean'], mean]], columns=['Date', 'Temperature', 'Mean', 'hPa'])
            )
        elif search:
            break
    
    if index_hpa % 1000 == 0:
        print(f'{index_hpa}/{len(dataframe_hpa)}')

In [67]:
dataframe_copy = dataframe_result.copy()
dataframe_copy.insert(3, 'hPa', 0)

In [ ]:
import math

for index, row in dataframe_copy.iterrows():
    hpa_row = dataframe_hpa.loc[dataframe_hpa['date'] == row['Date'].date()]
    mean = 0
    length = 0

    for index in range(1, len(hpa_row.iloc[0])):
        hpa_row.iloc[0][index] = float(hpa_row.iloc[0][index])
        if math.isnan(hpa_row.iloc[0][index]):
            length += 1
            mean += value

    dataframe_copy.at[index, 'hPa'] = mean / length

    if index % 5000 == 0:
        print(f'{index}/{len(dataframe_copy)}')


In [37]:
print(datetime.datetime.today().date())

2022-04-17
